In [1]:
%load_ext Cython

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as pyplot
import optuna as op
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Utils.createURM import createURM
from Utils.createICM import createICM
from Utils.combineURMICM import combine

### URM

In [3]:
URM = createURM()
ICM = createICM()

/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/createURM.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')
/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/createICM.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM = pd.read_csv('../Input/interactions_and_impressions.csv')


In [4]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [5]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2618 ( 6.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1592 ( 3.8%) Users that have less than 1 test interactions


In [6]:
URMICMCombined = combine(ICM=ICM, URM = URM)

### SLIM

In [7]:
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URMICMCombined.T, verbose=False)

def objective(trial):
    alpha = trial.suggest_float("alpha", 0, 1)
    l1_ratio = trial.suggest_float("l1_ratio", 0, 1)
    topK = trial.suggest_float("topK", 100, 500)
    #recommender.fit(alpha=alpha, l1_ratio=l1_ratio, topK=int(topK))
    recommender.fit()

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

In [8]:
study = op.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2022-11-04 22:44:23,244] A new study created in memory with name: no-name-5af1b769-bdbf-4c3f-88b7-1fb0a2aeb415
100%|█████████▉| 24504/24507 [19:29<00:00, 20.96it/s]


EvaluatorHoldout: Processed 39011 (100.0%) in 13.08 sec. Users per second: 2983


[I 2022-11-04 23:04:05,474] Trial 0 finished with value: 0.0 and parameters: {'alpha': 0.7429833064841921, 'l1_ratio': 0.45744303366055117, 'topK': 298.8563499865666}. Best is trial 0 with value: 0.0.
 94%|█████████▍| 23016/24507 [18:00<00:50, 29.48it/s][W 2022-11-04 23:22:09,578] Trial 1 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/Users/matteopancini/opt/anaconda3/envs/recsys/lib/python3.9/multiprocessing/pool.py", line 853, in next
    item = self._items.popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/matteopancini/opt/anaconda3/envs/recsys/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/tt/9v6608x550n51759gpcrffsc0000gn/T/ipykernel_12716/1651636419.py", line 8, in objective
    recommender.fit()
  File "/Users/matteopancini

KeyboardInterrupt: 

In [ ]:
alpha = study.best_params['alpha']
l1_ratio = study.best_params['l1_ratio']
topK = study.best_params['topK']

In [ ]:
alpha

In [ ]:
l1_ratio

In [ ]:
topK